<b>Name:</b> Natália Raymundi Pinheiro<br>
<b>Assistance:</b> Dr. Simon Birrer<br>
<b>Deadline:</b> February 26, 2025

In [ ]:
# Set path for ease of use
PATH = "/Users/nataliaraymundipinheiro/Documents/Stony Brook University/Semesters/Spring 2025/PHY688/modeling_project/"

# <u>Modeling Project:</u> Modeling a Real Hubble Space Telescope Image

<b>Chosen model:</b> [DESIJ0132-1600](https://github.com/AstroBridge/BDLensing/tree/main/lens_systems/DESIJ0132-1600), from [rafeee-adnan](https://github.com/rafee-adnan) on GitHub

## Imports

In [ ]:
import h5py
import matplotlib.pyplot as plt
import numpy as np

from lenstronomy

## Loading Data from `h5` Files

Load `kwargs_data` and `kwargs_psf` from `h5` files.

In [ ]:
with h5py.File(PATH + "model/data/DESIJ0132-1600/DESIJ0132-1600_F140W.h5", "r") as f:
    kwargs_data = {}
    for key in f:
        kwargs_data[key] = f[key][()]

kwargs_data

In [ ]:
with h5py.File(PATH + "model/data/DESIJ0132-1600/psf_F140W.h5", "r") as f:
    kwargs_psf = {}
    for key in f:
        kwargs_psf[key] = f[key][()]

kwargs_psf["psf_type"] = "PIXEL"
kwargs_psf

Extract `image_data` from `kwargs_data`. This will be saved as a `txt` file called `image`.

In [ ]:
image = kwargs_data["image_data"]

And, now, make the plot.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(20, 20))

axes[0].imshow(np.log10(image), origin="lower", cmap="cubehelix")
axes[0].set_title("Image")

axes[1].imshow(np.log10(image), origin="lower", cmap="cubehelix")
axes[1].set_title("Image with gridlines")
axes[1].grid(True, color="black")

plt.show()

## Building a Lens Model

The lens model is a union of three components: the lens galaxy's mass model, the lens galaxy's light model, and the source galaxy's light model. We give a list of profiles for each component.

In [ ]:
lens_model_list       = ["EPL", "SHEAR"]
lens_light_model_list = ["SERSIC_ELLIPSE", "SERSIC_ELLIPSE"]
source_model_list     = ["SHAPELETS"]

The `'EPL'` lens mass profile stands for Elliptical Power Law. The form of this convergence profile is given by:

$$\kappa(x, y) = \frac{3 - \gamma}{2} \left[ \frac{\theta_{\rm E}}{\sqrt{qx^2 + y^2 / q}} \right]^{\gamma - 1}$$

The position angle $\phi$ adjusts the orientation of the mass profile's major axis. The ellipticity parameters $q$ and $\phi$ can be reformulated as 
$$e_1 = \frac{1 - q}{1 + q} \cos 2\phi$$
$$e_2 = \frac{1 - q}{1 + q} \sin 2\phi$$

`lenstronomy` uses $e_1$ and $e_2$ instead of $q$ and $\phi$, because $e_1$ and $e_2$ are easier to handle in numerical optimization (for example, in MCMC).

Both the lens galaxy's and the source galaxy's light profiles are modeled with Sersic function, which is given by:

$$I(x, y) = I_{\rm e} \exp \left[ -b_{n} \left\{ \left( \frac{\sqrt{qx^2
+ y^2/q}}{R_{\rm Sersic}} \right)^{1/n_{\rm Sersic}} - 1 \right\}
\right]$$

## Lens Modeling with `lenstronomy`

### Lens Galaxy's Mass Model

#### `EPL` Parameters

In [ ]:
fixed_lens = []
kwargs_lens_init = []
kwargs_lens_sigma = []
kwargs_lower_lens = []
kwargs_upper_lens = []

fixed_lens.append({"gamma": 2})
kwargs_lens_init.append(
    {
        "theta_E": 1.4,
        "gamma": 2,
        "e1": -0.0749,
        "e2": -0.12077,
        "center_x": -0.2102,
        "center_y": -0.0083,
    }
)
kwargs_lens_sigma.append(
    {
        "theta_E": 0.1,
        "gamma": 0.1,
        "e1": 0.05,
        "e2": 0.05,
        "center_x": 0.5,
        "center_y": 0.5,
    }
)
kwargs_lower_lens.append(
    {
        "theta_E": 1,
        "gamma": 1.7,
        "e1": -0.5,
        "e2": -0.5,
        "center_x": -10,
        "center_y": -10,
    }
)
kwargs_upper_lens.append(
    {
        "theta_E": 10.0,
        "gamma": 2.1,
        "e1": 0.5,
        "e2": 0.5,
        "center_x": 10,
        "center_y": 10,
    }
)

#### SHEAR Parameters

In [ ]:
fixed_lens.append({"ra_0": 0, "dec_0": 0})

kwargs_lens_init.append({"gamma1": 0.066, "gamma2": -0.00263})
kwargs_lens_sigma.append({"gamma1": 0.01, "gamma2": 0.01})
kwargs_lower_lens.append({"gamma1": -0.3, "gamma2": -0.3})
kwargs_upper_lens.append({"gamma1": 0.3, "gamma2": 0.3})

lens_params = [
    kwargs_lens_init,
    kwargs_lens_sigma,
    fixed_lens,
    kwargs_lower_lens,
    kwargs_upper_lens,
]

### Source Galaxy's Light Model

#### Sérsic Ellipse Parameters

In [7]:
"""
fixed_source.append({"n_sersic": 1.0})
kwargs_source_init.append(
    {
        "R_sersic": 0.002,
        "n_sersic": 1,
        "e1": 0.9960,
        "e2": -0.1501,
        "center_x": -0.5247,
        "center_y": -0.0455,
        "amp": 1,
    }
)
kwargs_source_sigma.append(
    {
        "n_sersic": 0.5,
        "R_sersic": 0.001,
        "e1": 0.05,
        "e2": 0.05,
        "center_x": 0.2,
        "center_y": 0.2,
        "amp": 1,
    }
)
kwargs_lower_source.append(
    {
        "e1": -0.5,
        "e2": -0.5,
        "R_sersic": 0.001,
        "n_sersic": 0.5,
        "center_x": -10,
        "center_y": -10,
        "amp": 0,
    }
)
kwargs_upper_source.append(
    {
        "e1": 1,
        "e2": 1,
        "R_sersic": 0.2,
        "n_sersic": 5.0,
        "center_x": 10,
        "center_y": 10,
        "amp": 100,
    }
)
"""

'\nfixed_source.append({"n_sersic": 1.0})\nkwargs_source_init.append(\n    {\n        "R_sersic": 0.002,\n        "n_sersic": 1,\n        "e1": 0.9960,\n        "e2": -0.1501,\n        "center_x": -0.5247,\n        "center_y": -0.0455,\n        "amp": 1,\n    }\n)\nkwargs_source_sigma.append(\n    {\n        "n_sersic": 0.5,\n        "R_sersic": 0.001,\n        "e1": 0.05,\n        "e2": 0.05,\n        "center_x": 0.2,\n        "center_y": 0.2,\n        "amp": 1,\n    }\n)\nkwargs_lower_source.append(\n    {\n        "e1": -0.5,\n        "e2": -0.5,\n        "R_sersic": 0.001,\n        "n_sersic": 0.5,\n        "center_x": -10,\n        "center_y": -10,\n        "amp": 0,\n    }\n)\nkwargs_upper_source.append(\n    {\n        "e1": 1,\n        "e2": 1,\n        "R_sersic": 0.2,\n        "n_sersic": 5.0,\n        "center_x": 10,\n        "center_y": 10,\n        "amp": 100,\n    }\n)\n'

### SHAPELETS Parameters

In [8]:
fixed_source = []
kwargs_source_init = []
kwargs_source_sigma = []
kwargs_lower_source = []
kwargs_upper_source = []


fixed_source.append({"n_max": 10})

kwargs_source_init.append({"center_x": -0.5522, "center_y": 0.045511, "beta": 0.07})
kwargs_source_sigma.append({"center_x": 0.2, "center_y": 0.2, "beta": 0.001})
kwargs_lower_source.append({"center_x": -10, "center_y": -10, "beta": 0.001})
kwargs_upper_source.append({"center_x": 10, "center_y": 10, "beta": 0.2})

source_params = [
    kwargs_source_init,
    kwargs_source_sigma,
    fixed_source,
    kwargs_lower_source,
    kwargs_upper_source,
]

# joint_shapelets_with_sersic = [[0, 1, ["center_x", "center_y"]]]

### Lens Galaxy's Light Model

#### First SERSIC Ellipse Parameter

In [ ]:
fixed_lens_light = []
kwargs_lens_light_init = []
kwargs_lens_light_sigma = []
kwargs_lower_lens_light = []
kwargs_upper_lens_light = []

fixed_lens_light.append({"n_sersic": 1.0})

kwargs_lens_light_init.append(
    {
        "R_sersic": 1.8705,
        "n_sersic": 2,
        "e1": -0.1897,
        "e2": -0.115,
        "center_x": -0.255,
        "center_y": -0.0594,
        "amp": 16,
    }
)
kwargs_lens_light_sigma.append(
    {
        "n_sersic": 1,
        "R_sersic": 0.3,
        "e1": 0.05,
        "e2": 0.05,
        "center_x": 0.1,
        "center_y": 0.1,
        "amp": 1,
    }
)
kwargs_lower_lens_light.append(
    {
        "e1": -0.5,
        "e2": -0.5,
        "R_sersic": 0.001,
        "n_sersic": 0.5,
        "center_x": -10,
        "center_y": -10,
        "amp": 0,
    }
)
kwargs_upper_lens_light.append(
    {
        "e1": 0.5,
        "e2": 0.5,
        "R_sersic": 10,
        "n_sersic": 5.0,
        "center_x": 10,
        "center_y": 10,
        "amp": 100,
    }
)

#### Second SERSIC Ellipse PArameters

In [ ]:
fixed_lens_light.append({"n_sersic": 4.0})

kwargs_lens_light_init.append(
    {
        "R_sersic": 0.5280,
        "n_sersic": 2,
        "e1": -0.0355,
        "e2": -0.0266,
        "center_x": -0.2554,
        "center_y": -0.059,
        "amp": 16,
    }
)
kwargs_lens_light_sigma.append(
    {
        "n_sersic": 1,
        "R_sersic": 0.3,
        "e1": 0.05,
        "e2": 0.05,
        "center_x": 0.1,
        "center_y": 0.1,
        "amp": 1,
    }
)
kwargs_lower_lens_light.append(
    {
        "e1": -0.5,
        "e2": -0.5,
        "R_sersic": 0.001,
        "n_sersic": 0.5,
        "center_x": -10,
        "center_y": -10,
        "amp": 0,
    }
)
kwargs_upper_lens_light.append(
    {
        "e1": 0.5,
        "e2": 0.5,
        "R_sersic": 10,
        "n_sersic": 5.0,
        "center_x": 10,
        "center_y": 10,
        "amp": 100,
    }
)

joint_lens_light_with_lens_light = [[0, 1, ["center_x", "center_y", "e1", "e2"]]]

lens_light_params = [
    kwargs_lens_light_init,
    kwargs_lens_light_sigma,
    fixed_lens_light,
    kwargs_lower_lens_light,
    kwargs_upper_lens_light,
]